In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy Kubeflow Pipeline to Vertex Pipelines

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/06_model_training_pipeline.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/06_model_training_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/06_model_training_pipeline.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

For a rapid and reliable update of the pipelines in production, you need a robust automated CI/CD system. This automated CI/CD system lets your data scientists rapidly explore new ideas around feature engineering, model architecture, and hyperparameters. They can implement these ideas and automatically build, test, and deploy the new pipeline components to the target environment.

This noebook shows how to deploy a Kubeflow pipeline into VertexAI Pipeline using CI/CD.





### Objective

This notebook shows how to use Cloud Build to deploy your Kubeflow Pipeline into Vertex AI Pipeline. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

    * Deploy the Kubeflow Pipline, and run it on-demand
    * Deploy the new Kubeflow pipeline into the project using a Cloud Build trigger to automatically start a build whenever you make any changes to your source code. 


### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

In [ ]:
PIPELINE_NAME = f'fraud-finder-xgb-pipeline-{ID}'
PIPELINE_STORE_URI = f"gs://{BUCKET_NAME}/pipeline-store/"
IMAGE_REPOSITORY = f'fraudfinder-{ID}'
TRAINING_IMAGE_NAME='dask-xgb-classificator'
TRAINING_IMAGE_TAG='v1'
TRAINING_IMAGE_URI=f"us-central1-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REPOSITORY}/{TRAINING_IMAGE_NAME}:{TRAINING_IMAGE_TAG}"

In [ ]:
SUBSTITUTIONS=f"""\
_IMAGE_REPOSITORY='{IMAGE_REPOSITORY}',\
_PIPELINE_STORE_URI={PIPELINE_STORE_URI},\
_PIPELINE_NAME={PIPELINE_NAME},\
_WORKSPACE_DIR=./\
"""

!echo $SUBSTITUTIONS

### A) If you want to run CI/CD on-demand through the notebook

In [ ]:
!gcloud builds submit /home/jupyter/fraudfinder/vertex_ai/ --timeout=306m \
--config=deploy_kfp_pipeline/pipelinedeployment.yaml \
--machine-type=e2-highcpu-8 \
--substitutions {SUBSTITUTIONS}

In [ ]:
!gsutil ls $PIPELINE_STORE_URI

In [ ]:
!gsutil cp $pipelines_file_location ./pipelines/gcs/

### B) If you want to run CI/CD triggered based on source change in Github

A Cloud Build trigger automatically starts a build whenever you make any changes to your source code. You can configure the trigger to build your code on any changes to the source repository or only changes that match certain criteria.



In [ ]:
# TO DO: To integrate it with your repo
# !gcloud beta builds triggers create github \
# --name=triger-ff-training-pipeline-CICD \
# --repo-name=fraudfinder \
# --repo-owner=[to add] \
# --branch-pattern=main \
# --included-files=fraudfinder/vertexai/** \
# --build-config=fraudfinder/vertexai/cloudbuild.yaml \
# --substitutions= {SUBSTITUTIONS}